# Sequence Assembly

In [1]:
%load_ext autoreload
%autoreload 2

import set_dir

## Sequence Alignment

Here, I demonstrate the basic sequence alignment function.

In [2]:
from Sequence import Seq

In [3]:
s1 = Seq("GCATGCT")
s2 = Seq("GATTACA")

print( Seq.align(s1, s2) )

(0.0, [(<GCAT-GCT>, <G-ATTACA>), (<GCATG-CT>, <G-ATTACA>)])


By XOR-ing the outputs of an alignment where gaps are encouraged over mismatches, we can combine the two sequences into one:

In [4]:
score, (r1, r2) = Seq.align(s1, s2, gap_penalty=0, one_result=True)

print(score, r1, r2)
print(r1 ^ r2)

4.0 GCATGCT--- G-AT--TACA
GCATGCTACA


## Sequence Assembly

In this section, I define a sequence assembly algorithm using the `Sequence` module.

In [5]:
def pairs(arr, enumerate=False):
    """
    Generate all pairs of elements from a given list.
    
    If enumerate is true, return elements with their indices in the list.
    """
    for i in range(len(arr)):
        for j in range(i+1, len(arr)):
            if enumerate:
                yield (i, arr[i]), (j, arr[j])
            else:
                yield arr[i], arr[j]

In [6]:
def assemble_sequences(sequences, **kwargs):
    
    # Only use one result each round
    kwargs["one_result"]  = True
    
    # Set default values for alignments
    kwargs["gap_penalty"] = kwargs.get("gap_penalty", -1)
    kwargs["is_local"]    = kwargs.get("is_local",    True)
    kwargs["whole_seqs"]  = kwargs.get("whole_seqs",  True)
#     kwargs["score"]       = kwargs.get("score",      "simple")
    
    # Repeat until one sequence remains
    while len(sequences) > 1:

        # Initialize empty array for alignments
        alignments = []

        # Loop over each pair of sequences
        for (i, s1), (j, s2) in pairs(sequences, enumerate=True):
            
            # Compute the score of the optimal alignment, along with one
            # potential alignment that achieves that score
            score, (s1, s2) = Seq.align( s1, s2, **kwargs )
            
            # Keep track of score and sequence values, as well as their
            # indices in the original array
            alignments.append(( score, s1, s2, i, j ))
        
        # Print all the alignments
        print(f"Remaining Sequences: {len(sequences)}.")
        for score, s1, s2, i, j in alignments:
            print(f"  - Score: {score}")
            print(f"      {i} {s1}")
            print(f"      {j} {s2}")
            print("")
        print("")

        # Get the best alignment, by maximum score
        score, s1, s2, i, j = max( alignments )
        
        # Replace s1 and s2 with their alignment
        sequences.pop(j)
        sequences.pop(i)
        sequences.append(s1 ^ s2)
    
    return sequences[0]

In [7]:
s1 = Seq("TTCCGGAGAGGGAGCCTGAG")
s2 = Seq("GCCTGAGAAATGGCTACCACATC")
# s1 = Seq("TTCCGGAGAGGGAGCCTGAG")
# s2 = Seq("GCCTGAGAAATGGCTACCACATC")
# s1 = Seq("AAACC")
# s2 = Seq("CCTTTTT")
# s1 = Seq("AACCCAAAA")
# s2 = Seq("CCC")

# score, results = Seq.align(s1, s2, is_local=True, whole_seqs=True)
score, result = Seq.align(
    s1, s2, one_result=True, is_local=True, whole_seqs=True,
    score=(1, -10)
)
print(result[0])
print(result[1])
print("")

print(result[0] ^ result[1])
print("")

print(assemble_sequences([s1, s2]))

TTCCGGAGAGGGAGCCTGAG----------------
-------------GCCTGAGAAATGGCTACCACATC

TTCCGGAGAGGGAGCCTGAGAAATGGCTACCACATC

Remaining Sequences: 2.
  - Score: 7.0
      0 TTCCGGAGAGGGAGCCTGAG----------------
      1 -------------GCCTGAGAAATGGCTACCACATC


TTCCGGAGAGGGAGCCTGAGAAATGGCTACCACATC


My algorithm works as expected for short sequences:

In [8]:
seqs = [
    Seq("ACCCT"),
    Seq("AC"   ),
    Seq(   "CT")
]
assemble_sequences(seqs)

Remaining Sequences: 3.
  - Score: 2.0
      0 ACCCT
      1 AC---

  - Score: 2.0
      0 ACCCT
      2 ---CT

  - Score: 1.0
      1 AC-
      2 -CT


Remaining Sequences: 2.
  - Score: 2.0
      0 ---CT
      1 ACCCT




<ACCCT>

For longer sequences, it comes close to the expected solution.  Here, we expect the first three sequences to be placed in that order, with overlap at their ends, and the fourth to be a subsequence of the larger sequence.

In [9]:
seqs = [
    Seq("TTCCGGAGAGGGAGCCTGAG"),
    Seq("GCCTGAGAAATGGCTACCACATC"),
    Seq("CCACATCCACGGAGAGG"),
    Seq("CGGAGAGG"),
]

expected = "TTCCGGAGAGGGAGCCTGAGAAATGGCTACCACATCCACGGAGAGG"
full_seq = assemble_sequences(seqs, is_local=True, score=(1, -10))

print(f"Result:   {full_seq}")
print(f"Expected: {expected}")

Remaining Sequences: 4.
  - Score: 7.0
      0 TTCCGGAGAGGGAGCCTGAG----------------
      1 -------------GCCTGAGAAATGGCTACCACATC

  - Score: 8.0
      0 -----TTCC--GGAGAGGGAGCCTGAG
      2 -CCACATCCACGGAGAGG---------

  - Score: 8.0
      0 TTCCGGAGAGGGAGCCTGAG
      3 ---CGGAGAGG---------

  - Score: 7.0
      1 GCCTGAGAAATGGCTACCACATC----------
      2 ----------------CCACATCCACGGAGAGG

  - Score: 4.0
      1 --GCCTGAGAAATGGCTACCACATC--
      3 ----CGGAGAGG---------------

  - Score: 8.0
      2 CCACATCCACGGAGAGG
      3 ---------CGGAGAGG


Remaining Sequences: 3.
  - Score: 7.0
      0 GCCTGAGAAATGGCTACCACATC----------
      1 ----------------CCACATCCACGGAGAGG

  - Score: 7.0
      0 -------------GCCTGAGAAATGGCTACCACATC
      2 TTCCGGAGAGGGAGCCTGAG----------------

  - Score: 8.0
      1 -CCACATCCACGGAGAGG---------
      2 -----TTCC--GGAGAGGGAGCCTGAG


Remaining Sequences: 2.
  - Score: 7.0
      0 --------------------GCCTGAGAAATGGCTACCACATC
      1 -CCAC-TCCACGGAGAGGGAGCCTGAG------